In [1]:
using Base.Test
using Plots


In [2]:
abstract type RolledResult end

struct Success <: RolledResult
end

struct Critical <: RolledResult
end

struct Fumble <: RolledResult
end

import Base: +
+(r::RolledResult, ::Success) = r
+( ::RolledResult, ::Critical) = Critical()
+( ::RolledResult, ::Fumble) = Fumble()
+( ::Fumble,       ::Critical) = Success()
+( ::Critical,     ::Fumble) = Success()



+ (generic function with 201 methods)

In [3]:
@testset "result stacking" begin
    @test Critical()+Critical() == Critical()
    @test Fumble()+Critical() == Success()
    @test Fumble()+Success() == Fumble()
end

Test Summary:   | Pass  Total
result stacking |    3      3


Base.Test.DefaultTestSet("result stacking", Any[], 3, false)

In [4]:
"""
    rolltable(crit_range=1, fumble_range=1, nsides=6)

Returns a table of `RolledResult`s, based on the parameters given.
"""
function rolltable(crit_range=1, fumble_range=1, nsides=6)
    possible_results = Vector{RolledResult}(nsides)
    possible_results[:] .= Success()
    # I have defined addition on RolledResults for this:
    possible_results[1:fumble_range] .+= Fumble()
    possible_results[end-crit_range+1:end] .+= Critical()
    possible_results
end

rolltable

In [5]:
@testset "rolltable" begin
    @test rolltable()[1]==Fumble()
    @test rolltable()[end]==Critical()
    @test all(rolltable()[2:end-1].== Success())
    
    @test rolltable(2,2)== [Fumble(),Fumble(), Success(),Success(),Critical(), Critical()]
    
    #If crit range and fumble range overlap then they cancel out.
    @test rolltable(6,1)== [Success(),Critical(),Critical(),Critical(),Critical(), Critical()]
    @test rolltable(6,2)== [Success(),Success(),Critical(),Critical(),Critical(), Critical()]
end

Test Summary: | Pass  Total
rolltable     |    6      6


Base.Test.DefaultTestSet("rolltable", Any[], 6, false)

In [23]:
abstract type Mechanic end
struct Default <: Mechanic
end


In [24]:
function roll_skill_check(::Default, crit_range=1, fumble_range=1, nsides=6)
    result = rand(rolltable(crit_range, fumble_range, nsides))
    final_result(::Success) = 0
    final_result(::Fumble)  = -1
    final_result(::Critical) = +1
    final_result(result)
end

roll_skill_check (generic function with 8 methods)

In [25]:
roll_skill_check(Default(), 1,1)

0

In [50]:
function display_simulation(res)
    @show mean(res)
    @show std(res)
    @show minimum(res)
    @show quantile(res,0.25)
    @show median(res)
    @show quantile(res,0.75)
    @show maximum(res)
    histogram(res; xlims=(-3,3),bar_width=1, ylims=(0,length(res)))
end

display_simulation (generic function with 1 method)

In [43]:
display_simulation([roll_skill_check(Default(), 1,1) for _ in 1:10_000])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = -0.002
std(res) = 0.5739590080260866
minimum(res) = -1
quantile(res, 0.25) = 0.0
median(res) = 0.0
quantile(res, 0.75) = 0.0
maximum(res) = 1


## Now lets consider a different mechanic

Rather than a crit(/fumble) directly increasing(/decreasing) your result,
what if your rolled again such that a success would be an increase to yourresult.
and a crit(/fumble) would be another reroll.
Thus giving you the change to  double crit (/double fumble)

In [32]:
struct Explosions <: Mechanic
end

function roll_skill_check(::Explosions, crit_range=1, fumble_range=1, nsides=6)
    result = rand(rolltable(crit_range, fumble_range, nsides))
    final_result(::Success) = 0
    final_result(::Fumble)  = -1 + roll_skill_check(Explosions(), crit_range, fumble_range, nsides)
    final_result(::Critical) = 1 + roll_skill_check(Explosions(), crit_range, fumble_range, nsides)
    final_result(result)
end

roll_skill_check (generic function with 8 methods)

In [51]:
display_simulation([roll_skill_check(Explosions(), 1,1) for _ in 1:10_000])

mean(res) = 0.0123
std(res) = 0.7056900947660988
minimum(res) = -6
quantile(res, 0.25) = 0.0
median(res) = 0.0
quantile(res, 0.75) = 0.0
maximum(res) = 5


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [70]:
function display_contrast(crit_range, fumble_range, nsims=10_000)
    plt = display_simulation([roll_skill_check(Default(), crit_range, fumble_range) for _ in 1:nsims])
    title!(plt, "Default $(crit_range):$(fumble_range)")
    IJulia.display(plt)
    println("---------------- vs Explosions -------------")
    plt=display_simulation([roll_skill_check(Explosions(), crit_range, fumble_range) for _ in 1:nsims])
    title!(plt, "Explosions $(crit_range):$(fumble_range)")
    IJulia.display(plot!(plt))
end

display_contrast(1,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Default 1:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Explosions 1:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 0.0041
std(res) = 0.5705398685229257
minimum(res) = -1
quantile(res, 0.25) = 0.0
median(res) = 0.0
quantile(res, 0.75) = 0.0
maximum(res) = 1
---------------- vs Explosions -------------
mean(res) = 0.0057
std(res) = 0.7020803384799836
minimum(res) = -5
quantile(res, 0.25) = 0.0
median(res) = 0.0
quantile(res, 0.75) = 0.0
maximum(res) = 5


In [71]:
display_contrast(2,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Default 2:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 0.1593
std(res) = 0.6849601645663078
minimum(res) = -1
quantile(res, 0.25) = 0.0
median(res) = 0.0
quantile(res, 0.75) = 1.0
maximum(res) = 1


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Explosions 2:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

---------------- vs Explosions -------------
mean(res) = 0.3247
std(res) = 1.0355564435195077
minimum(res) = -5
quantile(res, 0.25) = 0.0
median(res) = 0.0
quantile(res, 0.75) = 1.0
maximum(res) = 8


In [72]:
display_contrast(3,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Default 3:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 0.3449
std(res) = 0.7382401302076342
minimum(res) = -1
quantile(res, 0.25) = 0.0
median(res) = 1.0
quantile(res, 0.75) = 1.0
maximum(res) = 1
---------------- vs Explosions -------------


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Explosions 3:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 1.0173
std(res) = 1.753655679095552
minimum(res) = -5
quantile(res, 0.25) = 0.0
median(res) = 1.0
quantile(res, 0.75) = 2.0
maximum(res) = 15


In [73]:
display_contrast(4,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Default 4:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 0.4978
std(res) = 0.7610867790919316
minimum(res) = -1
quantile(res, 0.25) = 0.0
median(res) = 1.0
quantile(res, 0.75) = 1.0
maximum(res) = 1
---------------- vs Explosions -------------


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Explosions 4:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 2.9526
std(res) = 3.6762079226383597
minimum(res) = -5
quantile(res, 0.25) = 0.0
median(res) = 2.0
quantile(res, 0.75) = 4.0
maximum(res) = 36


In [74]:
display_contrast(5,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Default 5:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 0.6636
std(res) = 0.748124995639706
minimum(res) = -1
quantile(res, 0.25) = 1.0
median(res) = 1.0
quantile(res, 0.75) = 1.0
maximum(res) = 1
---------------- vs Explosions -------------


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [75]:
rolltable(5, 1) #This is not going to workout, infinite rolls

6-element Array{RolledResult,1}:
 Fumble()  
 Critical()
 Critical()
 Critical()
 Critical()
 Critical()

In [76]:
display_contrast(6,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Default 6:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 Explosions 6:1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

mean(res) = 0.8335
std(res) = 0.3725474857825727
minimum(res) = 0
quantile(res, 0.25) = 1.0
median(res) = 1.0
quantile(res, 0.75) = 1.0
maximum(res) = 1
---------------- vs Explosions -------------
mean(res) = 4.9318
std(res) = 5.408315241124054
minimum(res) = 0
quantile(res, 0.25) = 1.0
median(res) = 3.0
quantile(res, 0.75) = 7.0
maximum(res) = 42
